# Mejorando El FrontEND

Vamos hacer un bind mount del directorio web-page, para que cada vez que hagamos un cambio en el folder **web-page**, se vea reflejado:

```sh
docker run --rm --name my-nginx-container -p 80:80 \
  --network platzinet \
  -v /home/carlos/Documents/tutienda-campesina-app/web-page:/usr/share/nginx/html:ro \
  my-nginx-proxy```

## Implementando un Formulario Basico.

Con su respectivo archivo de estilos y boton de submit. En JS se implementa el siguiente codigo, usando Jquery y mostrando los resultados obtenidos en consola:

```js
 $('#searchClientButton').click(function(e){ 
        e.preventDefault();
        counter++;
        console.log(counter);

        let client_id = $('#clientId').val();
        client_id = parseInt(client_id)
        console.log(`client_id: ${client_id} and type: ${typeof(client_id)}`);

        if(isNaN(client_id)){
            console.log('Debe ingresar un numero valido');
            return;
        }

        const api_URL = `http://localhost/api/client/${client_id}`;
        fetch(api_URL)
        .then(function(data){
            return data.json();
        })
        .then(function(client){
            console.log(client);
            console.log(client.DOB);
            console.log(client.address);
            console.log(client.lastName);
        })  
    });
```

![image](https://imgur.com/tzGdvI5.png)

## Mejorando 

La cuestion es que pasaria si hay por ejemplo un error en el servidor justo cuando se esta realizando una consulta, o que el usuario a buscar no exista en la BD. Sabemos que la API responde con _status code_.

### Renamed Parameter in First `.then()`: 

I changed `function(data)` to `function(response)` in the first `.then()` block. This is a common convention and makes it clearer that *response* is the full HTTP response object, not just the data payload.

When you make a network request using `fetch()`, like `fetch(api_URL)`, the first `.then()` block receives a `Response` object as its argument. This `Response` object is a representation of the entire HTTP response you get back from the server.

The `Response` object contains properties and methods that allow you to inspect the response and extract the data:

- `status`: The HTTP status code (e.g., `200` for OK, `404` for Not Found, 500 for Internal Server Error).
- `ok`: A boolean (true/false) indicating if the response was successful (status code 200-299).

Si lo que responde el servidor mediante el objeto `Response` es `not ok`, por ejemplo que el usuario no existe en la BD, un `404` por ejemplo, debe haber la forma de manipular este error. 

![image](https://imgur.com/02CyVGZ.png)

`return response.json().then(function(apiErrorResponse){ ... })`: If the response is not `ok`, the code proceeds to parse the response body as JSON

- `response.json()`: This method reads the response stream to completion and parses it as JSON. It returns a Promise

- `.then(function(apiErrorResponse){ ... })`: Once the JSON from the error response is successfully parsed, the callback function is executed, and the parsed JSON data is passed as `apiErrorResponse`

- `let error = new Error(error_message);`: A new Error object is created, using the `error_message` as its primary message. This standard JavaScript error object helps in consistent error handling.

- `error.status_code = response.status;` and `error.details = apiErrorResponse.detail.message;`: Custom properties (`status_code` and `details`) are added to the `error` object. This allows the `catch` block (further down in your original code) to access specific error information beyond just the generic error message, making debugging and user feedback more precise.

- `throw error;`: This is a crucial part. Instead of just logging or returning, `throw error;` explicitly throws the newly created `error` object. When an error is thrown within a Promise's `then` block, it rejects that Promise. This rejection is then caught by the `.catch() `block that follows the fetch chain, allowing for centralized error handling.

```py
        .catch(function(error){
            console.log('hola error catch...');  
            console.log(error_message);
            console.log(error.status_code, error.details);
        }) 
```
- This `.catch()` block is designed to handle any errors that occur during the fetch operation or in the preceding .then() blocks.

- `console.log(error.status_code, error.details);`: This logs the custom properties (`status_code`and `details`) that were added to the `error` object when it was thrown